## This notebook shows how to use get FABRIC tokens for Jupyterhub users

In [2]:
import os
from fabric_cli.cli.credential import CredMgr
from fabric_cli.cli.orchestrator import Orchestrator
import json

### Fabric Tokens
Fabric has 2 kinds of tokens:<br>
- Identity : required for Control/Measurement Framework APIs. Identity Token is valid upto an hour.
- Refresh : required to generate new Identity Tokens valid. Refresh Token is valid for 24 hours.

Fabric Identity token is required for Control/Measurement Framework APIs.<br>

When user logins to Jupyterhub after authenticating against CILogon, OIDC refresh token is derived. <br>
This token is available as the environment variable `CILOGON_REFRESH_TOKEN`.<br>

On the first login, we use `CILOGON_REFRESH_TOKEN` to generate new Fabric Identity Token and Fabric Refresh Token.<br>
For any subsequent use, we use Fabric Refresh Token. On every refresh, Fabric Refresh Token is changed and updated.<br>

NOTE: These steps are required for any experiments on Fabric Testbed.<br>

In [3]:
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

no stored variable or alias fabric_refresh_token
Stored 'fabric_refresh_token' (str)
Fabric Refresh Token https://cilogon.org/oauth2/refreshToken/2e7db1c1abdae5c8f64d9b1893accbb/1609861275809
CILOGON_REFRESH_TOKEN environment variable: https://cilogon.org/oauth2/refreshToken/2e7db1c1abdae5c8f64d9b1893accbb/1609861275809


### Get new Fabric Identity Token and update Fabric Refresh Token

In [4]:
try:
    refresh_res = CredMgr.refresh_token(project_name='all', scope='all', refresh_token=fabric_refresh_token)
    print("New Tokens received: {}".format(json.dumps(refresh_res)))
    fabric_id_token=refresh_res['id_token']
    fabric_refresh_token=refresh_res['refresh_token']
    print()
    print("New Refresh Token: {}".format(fabric_refresh_token))
    print()
    print("Stored new Refresh Token")
    %store fabric_refresh_token
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

New Tokens received: {"id_token": "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImI0MTUxNjcyMTExOTFlMmUwNWIyMmI1NGIxZDNiNzY2N2U3NjRhNzQ3NzIyMTg1ZTcyMmU1MmUxNDZmZTQzYWEifQ.eyJlbWFpbCI6Imt0aGFyZTEwQGVtYWlsLnVuYy5lZHUiLCJnaXZlbl9uYW1lIjoiS29tYWwiLCJmYW1pbHlfbmFtZSI6IlRoYXJlamEiLCJuYW1lIjoiS29tYWwgVGhhcmVqYSIsImlzcyI6Imh0dHBzOi8vY2lsb2dvbi5vcmciLCJzdWIiOiJodHRwOi8vY2lsb2dvbi5vcmcvc2VydmVyQS91c2Vycy8xMTkwNDEwMSIsImF1ZCI6ImNpbG9nb246L2NsaWVudF9pZC8yZjQ1NGU4YzUzMWJhMDNkOTczNGZhNjg3YzMyYTE5IiwidG9rZW5faWQiOiJodHRwczovL2NpbG9nb24ub3JnL29hdXRoMi9pZFRva2VuLzQ0YzNlZjQ4OGQ1MGY0NWRhODQwNTY0MjUxZDRjMWYzLzE2MDk4NjEyNzU1OTUiLCJhdXRoX3RpbWUiOiIxNjA5ODYxMjc1IiwiZXhwIjoxNjA5ODY0OTIxLCJpYXQiOjE2MDk4NjEzMjEsInByb2plY3RzIjp7fSwicm9sZXMiOlsicHJvamVjdC1sZWFkcyIsImFiZjAwMTRlLTcyZjUtNDRhYi1hYzYzLTVlYzVhNWRlYmJiOC1wbyIsImFiZjAwMTRlLTcyZjUtNDRhYi1hYzYzLTVlYzVhNWRlYmJiOC1wbSJdLCJzY29wZSI6ImFsbCJ9.ksQtAq6PBFE_E64MAqbDl9yPmv0jc7mJN27sjVekrjxiIk0rXPHoh8bImNnzLlru2fezckbd_q4iN43r8dMV3FPlFtQrUuvxbUtq9ZxJy2srNXqmYfPBhrnYG